In [1]:
# 라이브러리 모음
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# 모듈 모음
import modules as mds

In [2]:
# 누적 설비용량 데이터 정리

# 누적 / 연도별 데이터 불러오기
data_total = pd.read_excel("dataset/지역별_누적_설비용량.xlsx")
data_2024 = pd.read_excel("dataset/2024년도_신규_설비용량.xlsx")
data_2023 = pd.read_excel("dataset/2023년도_신규_설비용량.xlsx")
data_2022 = pd.read_excel("dataset/2022년도_신규_설비용량.xlsx")
data_2021 = pd.read_excel("dataset/2021년도_신규_설비용량.xlsx")

# 데이터프레임으로 변환
# ic = Installed Capacity = 설비용량
ic_total = pd.DataFrame(data_total)
ic_2024 = pd.DataFrame(data_2024)
ic_2023 = pd.DataFrame(data_2023)
ic_2022 = pd.DataFrame(data_2022)
ic_2021 = pd.DataFrame(data_2021)

# 숫자의 ',' 때문에 object 값이었던 데이터들을 int 값으로 변환
mds.delete_comma(ic_total)
mds.delete_comma(ic_2024)
mds.delete_comma(ic_2023)
mds.delete_comma(ic_2022)
mds.delete_comma(ic_2021)

# 컴럼 통일화를 위해 ic_total에 "합계" 컬럼 추가
ic_total["합계"] = ic_total.sum(axis=1, numeric_only=True)

# 하나의 DataFrame으로 병합
ic_total = pd.concat([ic_total, ic_2024, ic_2023, ic_2022, ic_2021], ignore_index=True)

# 각 연도별 누적값 계산
for i in range(1, len(ic_total)):
    ic_total.iloc[i, 1:] = ic_total.iloc[i-1, 1:] - ic_total.iloc[i, 1:]

# 연도 정리
ic_total.rename(columns={"Unnamed: 0": "연도"}, inplace=True)
ic_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 오름차순 정렬 (주석 처리 시 내림차순)
# ic_total.sort_values(by="연도", inplace=True)
# ic_total.reset_index(drop=True, inplace=True)

# 2020~2024 각 연도의 지역별 누적 설비용량
ic_total.head()

,연도,서울,부산,대구,인천,광주,대전,울산,세종,경기,충북,충남,전남,경북,경남,제주,강원,전북,합계
0,2024,50,275,420,163,353,65,152,88,2012,1613,3772,6023,3842,2016,621,1943,4590,27998
1,2023,50,234,339,139,303,55,122,78,1672,1408,3072,5410,3324,1745,610,1760,4304,24625
2,2022,48,202,297,124,274,50,99,70,1459,1240,2798,4568,2886,1564,587,1612,3995,21873
3,2021,44,170,265,110,249,46,87,62,1278,1065,2522,3911,2409,1347,549,1408,3357,18879
4,2020,41,144,200,100,215,39,69,55,1056,846,2038,3054,1771,1085,425,1105,2559,14802


In [ ]:
# 누적 발전량 데이터 정리

# 누적 / 연도별 데이터 불러오기
data_total_2022 = pd.read_excel("dataset/지역별_누적_발전량.xlsx")
data_2024 = pd.read_excel("dataset/2024년_신규_발전량.xlsx")
data_2023 = pd.read_excel("dataset/2023년_신규_발전량.xlsx")
data_2022 = pd.read_excel("dataset/2022년_신규_발전량.xlsx")
data_2021 = pd.read_excel("dataset/2021년_신규_발전량.xlsx")

# 데이터프레임으로 변환
# gen = Power Generation = 발전량
gen_total_2022 = pd.DataFrame(data_total_2022)
gen_2024 = pd.DataFrame(data_2024)
gen_2023 = pd.DataFrame(data_2023)
gen_2022 = pd.DataFrame(data_2022)
gen_2021 = pd.DataFrame(data_2021)

# 열과 행 전치
gen_2024 = gen_2024.T.reset_index(drop=True)
gen_2023 = gen_2023.T.reset_index(drop=True)
gen_2022 = gen_2022.T.reset_index(drop=True)
gen_2021 = gen_2021.T.reset_index(drop=True)

# 컴럼 정리
gen_2024.columns = gen_2024.iloc[0]
gen_2024.drop(labels=gen_2024.index[0], axis=0, inplace=True)
gen_2023.columns = gen_2023.iloc[0]
gen_2023.drop(labels=gen_2023.index[0], axis=0, inplace=True)
gen_2022.columns = gen_2022.iloc[0]
gen_2022.drop(labels=gen_2022.index[0], axis=0, inplace=True)
gen_2021.columns = gen_2021.iloc[0]
gen_2021.drop(labels=gen_2021.index[0], axis=0, inplace=True)


# 연도 컬럼 추가
gen_2024.insert(0, "연도", np.nan)
gen_2023.insert(0, "연도", np.nan)
gen_2022.insert(0, "연도", np.nan)
gen_2021.insert(0, "연도", np.nan)

# 숫자의 ',' 때문에 object 값이었던 데이터들을 int 값으로 변환
mds.delete_comma(gen_total_2022)
mds.delete_comma(gen_2024)
mds.delete_comma(gen_2023)
mds.delete_comma(gen_2022)
mds.delete_comma(gen_2021)

# 월별 데이터 연도별 데이터로 변환
gen_2024 = pd.DataFrame(gen_2024.sum()).T
gen_2023 = pd.DataFrame(gen_2023.sum()).T
gen_2022 = pd.DataFrame(gen_2022.sum()).T
gen_2021 = pd.DataFrame(gen_2021.sum()).T

# "합계" 컬럼 추가
gen_2024["합계"] = gen_2024.sum(axis=1, numeric_only=True)
gen_2023["합계"] = gen_2023.sum(axis=1, numeric_only=True)
gen_2022["합계"] = gen_2022.sum(axis=1, numeric_only=True)
gen_2021["합계"] = gen_2021.sum(axis=1, numeric_only=True)

# # 하나의 DataFrame으로 병합
gen_total = pd.concat([gen_2024, gen_2023, gen_total_2022, gen_2022, gen_2021], ignore_index=True)

# 연도 정리
gen_total = gen_total.drop(gen_total.columns[-1], axis=1)
gen_total.rename(columns={"지역": "연도"}, inplace=True)
gen_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 소수점 제거
gen_total = gen_total.astype(int)

# # 각 연도별 누적값 계산
gen_total.iloc[1, 1:] = gen_total.iloc[2, 1:] + gen_total.iloc[1, 1:] # 2023 계산
gen_total.iloc[0, 1:] = gen_total.iloc[1, 1:] + gen_total.iloc[0, 1:] # 2024 계산
gen_total.iloc[3, 1:] = gen_total.iloc[2, 1:] - gen_total.iloc[3, 1:] # 2021 계산
gen_total.iloc[4, 1:] = gen_total.iloc[3, 1:] - gen_total.iloc[4, 1:] # 2020 계산


# # 오름차순 정렬 (주석 처리 시 내림차순)
# # gen_total.sort_values(by="연도", inplace=True)
# # gen_total.reset_index(drop=True, inplace=True)

# # 2020~2024 각 연도의 지역별 누적 발전량
gen_total.head()

,연도,서울,부산,대구,인천,광주,대전,울산,세종,경기,충북,충남,전남,경북,경남,제주,강원,전북,합계
0,2024,454992,1584748,1962317,910178,2054262,372182,653470,524858,9209968,8280679,20170041,34738869,19298394,10914932,3712238,11167641,28815406,154825183
1,2023,397620,1296806,1518891,747103,1694113,305236,503506,437543,7143971,6533179,15870905,27619683,14957803,8711121,3077732,8984358,23235897,123035475
2,2022,341328,1056914,1166405,614164,1386507,249449,388527,358492,5478859,4990438,12385263,21236718,11154610,6771133,2429222,6964022,17956708,94928767
3,2021,284628,854401,851754,489619,1087788,196895,282818,287876,4001713,3603271,9087437,15534479,7826973,4971554,1763312,5086129,12875255,69085910
4,2020,232973,687334,598182,375544,837087,155021,195240,224741,2768139,2462795,6310846,10918067,5318784,3562221,1189794,3488999,9022902,48348677
